In [1]:
using JuMP
#using GLPKMathProgInterface
using CPLEX

# Usar callbacks

Una de las grandes ventajas de JuMP es la facilidad con la que podemos llamar a callbacks. Construiremos dos ejemplos.


## Uncapacitated lot-sizing problem

El modelo es el siguiente:

$\mbox{[MIN]} \sum_{t\in T} (K_t y_t + c_t x_t + h_t s_t )$ 

$s_{t-1}+x_t = s_t+d_t \qquad t\in T$

$0\leq x_t\leq My_t \qquad t\in T$

$x_t,s_t \geq 0\qquad t\in T$

$y_t \in \{0,1\}\qquad t\in T$

Hay un conjunto de desigualdades para este problema que cumplen:

\begin{equation}
\sum_{j\in L\setminus S}x_j + \sum_{j\in S}d_{jl} y_{j} \geq d_{1l},\qquad \forall L\in \{1,...,l\}, S\subseteq L
\end{equation}

(Explicación de la restricción en: http://www.dcc.fc.up.pt/~jpp/WOW/Wolsey-LS2.pdf página 16, podemos mirarlo como lo que se fabrica en unos periodos más lo máximo que se puede fabricar en otros debe cubrir la demanda hasta un periodo). Específicamente, conviene recordar que $d_{jl}=\sum_{t=j}^{l} d_t$.

Ejemplo tomado de: http://sbebo.github.io/blog/blog/2015/06/10/julia/

In [2]:
function separate(T::Int64, sumd::Array{Float64, 2}, y_val, x_val, y, x)
    TOL=1E-6
    S = zeros(Bool, T)
    for l in 1:T
        fill!(S, false)
        lhsvalue = 0.  #x(L\S) + sum{d[j:l]*y[j] for j in S}
        empty = true
        for j in 1:l
            if x_val[j] > sumd[j,l]*y_val[j] + TOL
                S[j] = true
                empty = false
                lhsvalue += sumd[j,l]*y_val[j]
            else
                lhsvalue += x_val[j]
            end
        end
        if empty #fuerza que haya algo en S, si no la igualdad es trivial
            continue
        end
        if lhsvalue < sumd[1,l] - TOL
            lhs = sum(x[1:l])
            for j = (1:T)[S]
                lhs += sumd[j,l]*y[j] - x[j]
            end
            return lhs - sumd[1,l] #para retornar mayor que 0
        end
    end
    return
end

function readULS(path::String)
    f = open(path,"r")
    T = parse(Int64,readline(f))
    c = map(float,split(strip(readline(f))))
    h = map(float,split(strip(readline(f))))
    K = map(float,split(strip(readline(f))))
    d = map(float,split(strip(readline(f))))
    close(f)
    T, c, h, K, d
end

function solveULS(path::String; solver=CplexSolver(), valid::Bool = true)

    T, c, h, K, d = readULS(path)
    m = Model(solver = solver)
    @variable(m, y[1:T], Bin)
    @variable(m, x[i = 1:T] >= 0)
    @variable(m, s[1:T] >= 0)

    @objective(m, Min, sum(c[t]*x[t] + K[t]*y[t] + h[t]*s[t] for t in 1:T))
    @constraint(m, activation[t = 1:T], x[t] <= sum(d[t:T])*y[t])
    @constraint(m, balance[t = 1:T], (t>1?s[t-1]:0) + x[t] == s[t] + d[t])

    #precompute sum(d[j:l])
    sumd = zeros(Float64, T, T)
    for l = 1:T, j = 1:l
        sumd[j,l] = sum(d[j:l])
    end
    
    separationtime = 0.
    separated = 0
    called = 0
    function lSgenerator(cb)
        tt = time()
        called += 1
        y_val = getvalue(y)
        x_val = getvalue(x)
     
        expr = separate(T, sumd, y_val, x_val, y, x)
        if expr != nothing
            @usercut(cb, expr >= 0)
            separated += 1
        end
        separationtime += time()-tt
    end
    if valid
        addcutcallback(m, lSgenerator)
    end
    status = solve(m)
    println("Objective value: ", getobjectivevalue(m))
    println("Separation time: $separationtime seconds")
    println("Separated: $separated")
    status
end

solveULS (generic function with 1 method)

In [3]:
run(`wget -O test.dat https://raw.githubusercontent.com/sbebo/julia-cuts/master/test.dat`)

--2017-10-30 18:08:00--  https://raw.githubusercontent.com/sbebo/julia-cuts/master/test.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1666 (1.6K) [text/plain]
Saving to: ‘test.dat’

     0K .                                                     100%  585M=0s

2017-10-30 18:08:01 (585 MB/s) - ‘test.dat’ saved [1666/1666]



In [4]:
run(`cat test.dat`)

200
1 4 1 2 4 4 1 4 1 3 1 4 2 1 4 4 1 1 2 4 2 2 4 2 2 1 4 1 4 1 3 4 1 1 4 1 3 4 1 3 1 2 4 3 4 3 1 4 1 2 4 2 3 1 2 2 4 2 1 3 2 2 4 1 1 2 2 1 4 1 3 1 2 3 2 1 4 3 2 2 2 2 4 4 3 1 3 4 2 3 2 1 1 1 1 1 1 4 1 3 3 3 1 1 2 3 3 3 3 2 3 2 2 2 4 1 3 4 1 3 2 2 4 4 1 1 2 3 4 4 4 3 1 1 2 2 3 3 1 4 1 1 4 1 4 1 2 1 4 3 3 2 2 3 4 2 3 4 1 4 2 4 4 3 3 1 4 4 2 4 1 4 2 1 3 3 4 3 3 4 4 1 4 4 4 3 1 1 3 2 2 1 1 1 4 3 3 3 3 2 
2 3 2 2 3 2 3 2 1 1 1 1 2 1 1 3 3 2 2 1 1 3 1 1 3 2 2 1 1 1 3 1 3 3 1 3 1 1 2 1 1 1 1 1 1 3 2 1 2 3 3 3 3 2 2 3 3 2 3 1 3 2 3 2 1 2 2 3 1 1 2 2 2 1 3 3 3 3 2 3 3 1 1 1 2 2 2 2 1 1 3 1 1 2 3 2 2 2 1 1 2 1 1 3 2 1 3 2 2 3 2 3 1 1 2 1 1 2 3 3 3 2 1 2 2 1 1 3 2 1 2 3 1 3 1 3 2 2 1 3 1 1 1 3 1 3 3 2 1 2 2 1 2 1 3 2 3 2 3 1 2 2 3 3 1 2 2 3 3 3 1 1 3 1 3 2 2 3 2 1 2 3 2 1 1 2 1 3 1 1 3 2 2 1 2 3 1 1 2 1 
10 8 6 6 10 8 9 5 10 10 5 7 8 10 5 7 6 5 8 6 8 10 9 6 9 10 5 5 10 6 9 8 10 10 10 10 7 5 9 7 10 8 9 10 8 7 9 7 10 10 8 9 5 9 5 8 8 10 5 9 5 5 7 9 9 9 9 7 7 7 10 5 5 7 9 9 6 7 7 7 9 6 8 8 8 10 9 6

In [5]:
#solveULS("test.dat",solver=CplexSolver(CPX_PARAM_CUTSFACTOR=1,CPX_PARAM_SCRIND=0),valid=false)
solveULS("test.dat",solver=CplexSolver(CPX_PARAM_CUTSFACTOR=1,CPX_PARAM_SCRIND=1),valid=true)

CPXPARAM_Preprocessing_Linear                    0
CPXPARAM_MIP_Limits_CutsFactor                   1
CPXPARAM_MIP_Strategy_CallbackReducedLP          0
Tried aggregator 2 times.
MIP Presolve eliminated 2 rows and 2 columns.
Aggregator did 1 substitutions.
Reduced MIP has 397 rows, 597 columns, and 993 nonzeros.
Reduced MIP has 199 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.63 ticks)
Found incumbent of value 2.9264526e+07 after 0.06 sec. (1.15 ticks)
Probing time = 0.00 sec. (2.32 ticks)
Tried aggregator 1 time.
Reduced MIP has 397 rows, 597 columns, and 993 nonzeros.
Reduced MIP has 199 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.61 ticks)
Probing time = 0.00 sec. (2.22 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.01 sec. (0.81 ticks)

        Nodes                                         Cu

:Optimal

## Introducir una heurística

El ejemplo anterior hace uso de las "cut callback", ahora vamos a ver cómo introducir una heurística. Repetiremos gran parte del código del ejemplo anterior y usaremos una heurística "simple" (retornaremos a las posibles soluciones a este problema cuando veamos programación dinámica)

La heurística evaluará las variables $y$ desde el último instante hasta el primero. Cada vez que tengamos un acumulado mayor a 1 haremos un "trigger" de pedido y compraremos la suma de las demandas hasta el siguiente periodo de pedido. Esta heurística no es un método recomendado para resolver el problema (el problema es fácil como se verá al final de estas sesiones), pero se incluye como ejemplo

In [43]:
function solveULSwithHeuristic(path::String; solver=CplexSolver(), valid::Bool = true)

    T, c, h, K, d = readULS(path)
    m = Model(solver = solver)
    @variable(m, y[1:T], Bin)
    @variable(m, x[i = 1:T] >= 0)
    @variable(m, s[1:T] >= 0)

    @objective(m, Min, sum(c[t]*x[t] + K[t]*y[t] + h[t]*s[t] for t in 1:T))
    @constraint(m, activation[t = 1:T], x[t] <= sum(d[t:T])*y[t])
    @constraint(m, balance[t = 1:T], (t>1?s[t-1]:0) + x[t] == s[t] + d[t])

    #precompute sum(d[j:l])
    sumd = zeros(Float64, T, T)
    for l = 1:T, j = 1:l
        sumd[j,l] = sum(d[j:l])
    end
    
    separationtime = 0.
    separated = 0
    called = 0
    function lSgenerator(cb)
        tt = time()
        called += 1
        y_val = getvalue(y)
        x_val = getvalue(x)
     
        expr = separate(T, sumd, y_val, x_val, y, x)
        if expr != nothing
            @usercut(cb, expr >= 0)
            separated += 1
        end
        separationtime += time()-tt
    end
    function heuristic(cb)
        y_val = getvalue(y)
        y_calc=zeros(Int64,T)
        x_calc=zeros(Float64,T)
        #println("In heuristic y=$y_val")
        yAc=0.0
        t=T
        while t>=1
            yAc += y_val[t]
            #println("(",t,",",y_val[t],",",yAc,") ")
            if yAc > 0.999
                yAc -= 1.0
                y_calc[t]=1
            end
            t -= 1
        end
        #println("\n")
        y_calc[1]=1
        xAc=0
        t=T
        while t>=1
            xAc += d[t]
            if y_calc[t] == 1
                x_calc[t]=xAc
                xAc=0
            end
            #println(t," .. ",y_calc[t],"\t",d[t],"\t",xAc,"\t",x_calc[t],"\n")
            t -= 1
        end
        #println("y_calc=$y_calc x_calc=$x_calc")
        for t in 1:T
            setsolutionvalue(cb, y[t], y_calc[t])
        end
        for t in 1:T
            setsolutionvalue(cb, x[t], x_calc[t])
        end
        addsolution(cb)
    end
    if valid
        addcutcallback(m, lSgenerator)
        addheuristiccallback(m, heuristic)
    end

    status = solve(m)
    println("Objective value: ", getobjectivevalue(m))
    println("Separation time: $separationtime seconds")
    println("Separated: $separated")
    status
end

solveULSwithHeuristic (generic function with 1 method)

In [44]:
solveULSwithHeuristic("test.dat",solver=CplexSolver(CPX_PARAM_CUTSFACTOR=1,CPX_PARAM_SCRIND=1),valid=true)

CPXPARAM_Preprocessing_Linear                    0
CPXPARAM_MIP_Limits_CutsFactor                   1
CPXPARAM_MIP_Strategy_CallbackReducedLP          0
Tried aggregator 2 times.
MIP Presolve eliminated 2 rows and 2 columns.
Aggregator did 1 substitutions.
Reduced MIP has 397 rows, 597 columns, and 993 nonzeros.
Reduced MIP has 199 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.63 ticks)
Found incumbent of value 2.9264526e+07 after 0.00 sec. (1.15 ticks)
Probing time = 0.00 sec. (2.32 ticks)
Tried aggregator 1 time.
Reduced MIP has 397 rows, 597 columns, and 993 nonzeros.
Reduced MIP has 199 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.61 ticks)
Probing time = 0.00 sec. (2.22 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.81 ticks)

        Nodes                                         Cu

:Optimal

## Absolute MMR Assignment Problem

Basado en Jordi Pereira, Igor Averbakh, Exact and heuristic algorithms for the interval data robust assignment problem, In Computers & Operations Research, Volume 38, Issue 8, 2011, Pages 1153-1163, ISSN 0305-0548, https://doi.org/10.1016/j.cor.2010.11.009.

Utilizaremos el siguiente modelo

$\min \sum_{(u,v)\in E} c_{uv}^+ x_{uv} - \theta$


s.t.

$\theta \leq \sum_{(u,v)\in E} c_{uv}^- y_{uv} + \sum_{(u,v)\in E} y_{uv} (c_{uv}^+ - c_{uv}^-) x_{uv}\qquad \forall Y\in A$

$\sum_{u\in U} x_{uv} = 1, \qquad v\in V$;

$\sum_{v\in V} x_{uv} = 1, \qquad u\in U$;

$x_{uv}\in \{0,1\}, \qquad u\in U, v\in V$.

Nótese que la restricción que calcula $\theta$ hace uso de diversas propiedades del problema (sabemos que el escenario que maximiza la diferencia entre dos soluciones es un escenario extremo de características concretas y que una de esas soluciones define la peor alternativa y por tanto el worst-case regret de una solución $X$.

En realidad es más conveniente implementar como:

$\min \theta$


s.t.

$\theta \geq \sum_{(u,v)\in E} c_{uv}^s x_{uv} - c_{uv}^s x_{uv}^{\ast}\forall s\in S$

y específicamente definir $s$ para una solución $x$ como su caso extremo. Esto es, $c_{uv}^s= c_{uv}^+$ si $x_{uv}=1$ y $c_{uv}^s= c_{uv}^-$ en el resto de los casos.

In [46]:
function doInstanciaAleatoria(n::Int64)
    cminus = rand(1:1000000, n*n)
    cplus=map(x -> rand(x:x+1000000), cminus)
    return reshape(cminus,n,n),reshape(cplus,n,n)
end

doInstanciaAleatoria (generic function with 1 method)

In [86]:
srand(0)
cminus,cplus=doInstanciaAleatoria(25)

(
[50241 928179 … 924002 311082; 498707 723431 … 989592 967818; … ; 677683 755918 … 359544 786259; 988872 976708 … 559808 153175],

[890278 1325150 … 1067118 1179363; 1022298 1015118 … 1238138 1085748; … ; 891074 1341305 … 1296899 1104472; 1555761 1881310 … 946543 191407])

In [48]:
function solveAP(c,n::Int64,solver)
    submodel = Model(solver = solver)
    @variable(submodel, x[1:n,1:n],Bin)
    @objective(submodel,Min, sum(c[i,j]*x[i,j] for i in 1:n, j in 1:n) )
    
    @constraint(submodel, origin[i = 1:n], sum(x[i,1:n]) == 1)
    @constraint(submodel, destination[j = 1:n], sum(x[1:n,j]) == 1)
    status = solve(submodel)
    #print("solucion: ",status,"\n\n\n")
    valor=getobjectivevalue(submodel)
    return valor
end

function solveRAP(cminus, cplus, n::Int64, solver=CplexSolver())
    m = Model(solver = solver)
    
    @variable(m, x[1:n,1:n], Bin)
    @variable(m,θ)

    @objective(m, Min, θ )
    
    @constraint(m, origin[i = 1:n], sum(x[i,1:n]) == 1)
    @constraint(m, destination[j = 1:n], sum(x[1:n,j]) == 1)
    
    cMidPoint=zeros(Float64,n,n)
    for i in 1:n
        for j in 1:n
            cMidPoint[i,j]=cminus[i,j]+cplus[i,j]
        end
    end
    optimalCost = solveAP(cMidPoint,n,CplexSolver(CPX_PARAM_SCRIND=0))
    rhs = - optimalCost 
    for i in 1:n
        for j in 1:n
            rhs += cMidPoint[i,j]*x[i,j]
        end
    end
    @constraint(m,θ >=rhs)        
    #print(m)

    tt = 0.0
    called = 0.0
    separationtime=0.0
    separated=0
    
    
    function cutGenerator(m)
        #print("\n\n entro en generador de cortes")
        EPSILON=0.00001
        tt = time()
        called += 1
        x_val = getvalue(x)
                
        for i in 1:n
            for j in 1:n
                if x_val[i,j]>EPSILON && x_val[i,j]<(1.0-EPSILON)
                    return
                end
            end
        end
        # tenemos una solucion entera, buscamos los costos
        cost=deepcopy(cminus)
        for i in 1:n
            for j in 1:n
                if x_val[i,j]>EPSILON
                    cost[i,j]=cplus[i,j]
                end
            end
        end
        optimalCost = solveAP(cost,n,CplexSolver(CPX_PARAM_SCRIND=0))
        #println("\t\t",optimalCost,"\n")
       
        rhs = - optimalCost 
        for i in 1:n
            for j in 1:n
                rhs += cost[i,j]*x[i,j]
            end
        end
        #println(rhs)
        #print("*")
        @lazyconstraint(m, θ >= rhs )
        separated += 1
        separationtime += time()-tt
    end
    addlazycallback(m, cutGenerator) #es necesario un corte lazy ya que le indica al programa que no la he incluido

    status = solve(m)
    println(status)
    println("maxRegret: ",getobjectivevalue(m))
end

solveRAP (generic function with 2 methods)

In [85]:
solveRAP(cminus,cplus,25)

LoadError: BoundsError: attempt to access 10×10 Array{Int64,2} at index [1,11]

## Compliquemos ligeramente el ejemplo anterior

Tenemos dos puntos que podríamos mejorar:

(1) No estamos generando cortes para las soluciones fraccionales

(2) Las soluciones fraccionales permitirían incorporar nuevas soluciones heurísticas en el modelo

El primer punto es relativamente simple de solucionar. Podemos construir una solución "integralizada" y usarla para generar un corte.

El segundo punto es más interesante desde un punto de vista de implementación porque no es "trivial" compartir información encontrada durante las llamadas a cortes y las llamadas a la heurística. La solución más fácil es compartir una lista de soluciones evaluadas en los cortes y después introducir esas soluciones en la heurística (por ejemplo, mediante un stack de vectores)

Pregunta "tricky". ¿Por qué no era necesario antes?

In [ ]:
function solveRAPwithHeuristic(cminus, cplus, n::Int64, solver=CplexSolver())
    m = Model(solver = solver)
    
    @variable(m, x[1:n,1:n], Bin)
    @variable(m,θ)

    @objective(m, Min, θ )
    
    @constraint(m, origin[i = 1:n], sum(x[i,1:n]) == 1)
    @constraint(m, destination[j = 1:n], sum(x[1:n,j]) == 1)
    
    cMidPoint=zeros(Float64,n,n)
    for i in 1:n
        for j in 1:n
            cMidPoint[i,j]=cminus[i,j]+cplus[i,j]
        end
    end
    optimalCost = solveAP(cMidPoint,n,CplexSolver(CPX_PARAM_SCRIND=0))
    rhs = - optimalCost 
    for i in 1:n
        for j in 1:n
            rhs += cMidPoint[i,j]*x[i,j]
        end
    end
    @constraint(m,θ >=rhs)        
    #print(m)

    tt = 0.0
    called = 0.0
    separationtime=0.0
    separated=0
    pending =Set()

    function heuristic(m)
        #println("\nheuristic")
        #if !isempty(pending)
        #    println("not empty")
        #end
    end
    function cutGenerator(m)
        print("\n\n entro en generador de cortes")
        heuristic=false
        EPSILON=0.00001
        tt = time()
        called += 1
        x_val = getvalue(x)
        println(x_val)        
        for i in 1:n
            for j in 1:n
                if x_val[i,j]>EPSILON && x_val[i,j]<(1.0-EPSILON)
                    heuristic=true
                end
            end
        end
        if heuristic
            xTest=zeros(Int64,n)
            yUsed=zeros(Int64,n)
            bestVal=0
            for i in 1:n
                val=0
                bestVal=0
                for j in 1:n
                    if yUsed[j]==0 && val<x_val[i,j]
                        bestVal=j
                    end
                end
                xTest=j
                yUsed[j]=1
            end
            for i in 1:n
                for j in 1:n
                    x_val[i,j]=0
                end
                x_val[i,xTest]=1
            end
        end
        # tenemos una solucion entera, buscamos los costos
        print("\t\t\t",heuristic,"\t\t\t")
        if heuristic
            print(x_val)
        end
        print("\n")
        cost=deepcopy(cminus)
        for i in 1:n
            for j in 1:n
                if x_val[i,j]>EPSILON
                    cost[i,j]=cplus[i,j]
                end
            end
        end
        optimalCost = solveAP(cost,n,CplexSolver(CPX_PARAM_SCRIND=0))
        #println("\t\t",optimalCost,"\n")
        
        rhs = - optimalCost 
        for i in 1:n
            for j in 1:n
                rhs += cost[i,j]*x[i,j]
            end
        end
        #println(rhs)
        #print("*")
        @lazyconstraint(m, θ >= rhs )
        separated += 1
        separationtime += time()-tt
    end
    addlazycallback(m, cutGenerator) #es necesario un corte lazy ya que le indica al programa que no la he incluido
    addusercut(m, cutGenerator) #es necesario un corte lazy ya que le indica al programa que no la he incluido
    addheuristiccallback(m,heuristic)
    status = solve(m)
    println(status)
    println("maxRegret: ",getobjectivevalue(m))
end

In [ ]:
solveRAPwithHeuristic(cminus,cplus,25)